# Preparation of the analysis
## Importing the libraries

In [2]:
import pandas as pd
import os
import tarfile
import gzip
import datetime
import plotly.express as px 
import plotly.graph_objects as go

!pwd
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

/content
Mounted at /content/drive


## Importing the dataset

We first define the path to the dataset.

In [3]:
PATH_BA = '/content/drive/MyDrive/ada-2022-projet-datalcoholic/BeerAdvocate_CSV.tar.gz'
PATH_RB = '/content/drive/MyDrive/ada-2022-projet-datalcoholic/RateBeer_CSV.tar.gz'
folder_BA = tarfile.open(PATH_BA)
folder_RB = tarfile.open(PATH_RB)

We now import the dataset and convert the different .csv files to pandas dataframes. This is done on both BeerAdvocate and RateBeer datasets. For each dataset we obtain four dataframes: one for the reviews, one for the beers, one for the breweries and one for the users.

In [4]:
# Extracting the files for BeerAdvocate
df_beers_BA = pd.read_csv(folder_BA.extractfile(folder_BA.getmember('../Data/BeerAdvocate/beers.csv')))
df_users_BA = pd.read_csv(folder_BA.extractfile(folder_BA.getmember('../Data/BeerAdvocate/users.csv')))
df_reviews_BA = pd.read_csv(folder_BA.extractfile(folder_BA.getmember('../Data/BeerAdvocate/reviews.csv')))
df_breweries_BA = pd.read_csv(folder_BA.extractfile(folder_BA.getmember('../Data/BeerAdvocate/breweries.csv')))

# Extracting the files for RateBeer
df_beers_RB = pd.read_csv(folder_RB.extractfile(folder_RB.getmember('../Data/RateBeer/beers.csv')))
df_users_RB = pd.read_csv(folder_RB.extractfile(folder_RB.getmember('../Data/RateBeer/users.csv')))
df_reviews_RB = pd.read_csv(folder_RB.extractfile(folder_RB.getmember('../Data/RateBeer/reviews.csv')))
df_breweries_RB = pd.read_csv(folder_RB.extractfile(folder_RB.getmember('../Data/RateBeer/breweries.csv')))

## Preprocessing the dataframes

### Adding location information to the dataframes

For the review dataframes of both datasets, we want to add two columns that corresponds to the country of the user and the country of the brewery. We do this by extracting the information from the user and brewery dataframes and merging them with the review dataframe. In the end, we obtain `df_RB` and `df_BA` which have the reviews of RateBeer and BeerAdvocate respectively.

In [5]:
#RateBeer
user_nat_RB=pd.DataFrame()
user_nat_RB['user_name']=df_users_RB['user_name']
user_nat_RB['location_user']=df_users_RB['location']
new_reviews_RB=pd.merge(df_reviews_RB, user_nat_RB,  how='inner', on='user_name')

beers_nat_RB=pd.DataFrame()
beers_nat_RB['beer_id']=df_breweries_RB['id']
beers_nat_RB['beers_location']=df_breweries_RB['location']
new_reviews_RB['beer_id']=new_reviews_RB['beer_id'].apply(lambda x: int(x))
df_RB= pd.merge(new_reviews_RB, beers_nat_RB, how='inner', on='beer_id')

#BeerAdvocate
user_nat_BA=pd.DataFrame()
user_nat_BA['user_name']=df_users_BA['user_name']
user_nat_BA['location_user']=df_users_BA['location']
new_reviews_BA=pd.merge(df_reviews_BA, user_nat_BA,  how='inner', on='user_name')

beers_nat_BA=pd.DataFrame()
beers_nat_BA['beer_id']=df_breweries_BA['id']
beers_nat_BA['beers_location']=df_breweries_BA['location']
new_reviews_BA['beer_id']=new_reviews_BA['beer_id'].apply(lambda x: int(x))
df_BA = pd.merge(new_reviews_BA, beers_nat_BA, how='inner', on='beer_id')

#### Adding states for the US based locations

The geographical information contain the name of the country for users and breweries. However, for the US based locations, we also have the name of the state. We want to extract this information and add it to the dataframe. To do so we create will create an extra column for both breweries and users that will contain the US postal abbreviations for each state.

We first extract the postal abbreviations for each state from wikipedia using `pd.read_html`. 

We then process the dataframe into one (`US_states`) containing a column for the postal abbreviations and another one for the corresponding state names. We create from `US_states` two dataframes: `US_states_user` and `US_states_beer`. 

We add the corresponding postal abbreviation to the user and brewery dataframes. We do this by merging the `df_BA` and `df_RB` dataframes with `US_states_user` and `US_states_beer` respectively. 

We then finish processing the locations by dropping the state name in the location column.


In [6]:
# Creating the US_states dataframe
US_states = pd.read_html('https://en.wikipedia.org/wiki/ISO_3166-2:US')[0] 
US_states['Subdivision name (en)'] = US_states['Subdivision name (en)'].apply(lambda x: 'United States, ' + x) 
US_states['Code'] = US_states['Code'].apply(lambda x: x[3:]) 
US_states.drop(columns=['Subdivision category'], inplace=True) 

# Creating the two dataframes from the US_states dataframe
US_states_user=US_states.rename(columns={'Subdivision name (en)':'location_user', 'Code':'US_Code_User'}) 
US_states_beer=US_states.rename(columns={'Subdivision name (en)':'beers_location', 'Code':'US_Code_Beer'}) 

# Merging to add the postal abbreviations to the RateBeer and BeerAdvocate dataframes
df_BA=pd.merge(US_states_beer, df_BA, how='outer', on='beers_location') 
df_BA=pd.merge(US_states_user, df_BA, how='outer', on='location_user') 


df_RB=pd.merge(US_states_beer, df_RB, how='outer', on='beers_location')
df_RB=pd.merge(US_states_user, df_RB, how='outer', on='location_user')

def keep_United_States_if_in_the_string(x): 
    if 'United States' in x: 
        return 'United States' 
    else: 
        return x 

# for the location and nationalities we kept only 'United States' and removed the State name after the comma for ploting.

df_BA['beers_location']=df_BA['beers_location'].apply(lambda x: str(x)) 
df_BA['beers_location']=df_BA['beers_location'].apply(lambda x: keep_United_States_if_in_the_string(x)) 
df_BA['location_user']=df_BA['location_user'].apply(lambda x: str(x)) 
df_BA['location_user']=df_BA['location_user'].apply(lambda x: keep_United_States_if_in_the_string(x)) 

df_RB['beers_location']=df_RB['beers_location'].apply(lambda x: str(x))
df_RB['beers_location']=df_RB['beers_location'].apply(lambda x: keep_United_States_if_in_the_string(x))
df_RB['location_user']=df_RB['location_user'].apply(lambda x: str(x))
df_RB['location_user']=df_RB['location_user'].apply(lambda x: keep_United_States_if_in_the_string(x))


### Exploring where users come from

In order to better define our analysis, we will first explore the geographical distribution of the users. To do so, we will first create a dataframe containing the number of users per country. We will then plot the distribution of the users on a world map. We do this for both dataframes `df_BA` and `df_RB`.

In [7]:
count_country_users_RB=df_RB.groupby('location_user').count()['user_name'].to_frame().reset_index()
count_country_users_RB=count_country_users_RB.rename(columns={'user_name':'count_users'})

fig = px.choropleth(count_country_users_RB, 
                    locations='location_user',  
                    locationmode='country names',  
                    scope="world", 
                    color='count_users', 
                    )
fig.update_layout(title_text='RateBeer - Number of users per country') 
fig.show()

In [35]:
count_country_users_BA=df_BA.groupby('location_user').count()['user_name'].to_frame().reset_index()
count_country_users_BA=count_country_users_BA.rename(columns={'user_name':'count_users'})

fig = px.choropleth(count_country_users_BA, 
                    locations='location_user',  
                    locationmode='country names',  
                    scope="world", 
                    color='count_users',  
                    )
fig.update_layout(title_text='BeerAdvocate - Number of users per country') 
fig.show()

### Focus on the US

We want to focus on the US for our analysis. We therefore create two new dataframes `BA_US` and `RB_US` that contain only the reviews of the US based users.

In [36]:
RB_US = df_RB[df_RB['location_user'] == 'United States']
BA_US = df_BA[df_BA['location_user'] == 'United States']

Let's quickly check if some states have very few reviews.

In [ ]:
RB_US['US_Code_User'].value_counts()[RB_US['US_Code_User'].value_counts() < 1000]

HI    831
SD    787
WY    736
DC      1
AS      1
GU      1
MP      1
PR      1
UM      1
VI      1
Name: US_Code_User, dtype: int64

In [ ]:
BA_US['US_Code_User'].value_counts()[BA_US['US_Code_User'].value_counts() < 1000]

MT    896
UT    570
ND    456
SD    261
DC      1
AS      1
GU      1
MP      1
PR      1
UM      1
VI      1
Name: US_Code_User, dtype: int64

As we can see the US territories have 1 review each. We will therefore remove them from the dataframe since they will not be useful for our analysis.

In [ ]:
RB_US = RB_US[~RB_US['US_Code_User'].isin(['DC', 'AS', 'GU', 'MP', 'PR', 'VI', 'UM'])]
BA_US = BA_US[~BA_US['US_Code_User'].isin(['DC', 'AS', 'GU', 'MP', 'PR', 'VI', 'UM'])]

Let's now explore the distribution of the users in the US. We will plot the distribution of the users on a map of the US.

In [38]:
count_state_users_RB=RB_US.groupby('US_Code_User').count()['user_name'].to_frame().reset_index()
count_state_users_RB=count_state_users_RB.rename(columns={'user_name':'count_users'})

fig = px.choropleth(count_state_users_RB, 
                    locations='US_Code_User',  
                    locationmode='USA-states',  
                    scope="usa", 
                    color='count_users',  
                    )
fig.update_layout(title_text='RateBeer - Number of users per states') 

fig.show()
#comment the code below if you don't want to generate the html
fig.write_html('map_users_RB.html', include_plotlyjs='cdn', full_html=False, config={'displayModeBar': False})

In [39]:
count_state_users_BA=BA_US.groupby('US_Code_User').count()['user_name'].to_frame().reset_index()
count_state_users_BA=count_state_users_BA.rename(columns={'user_name':'count_users'})

fig = px.choropleth(count_state_users_BA, 
                    locations='US_Code_User',  
                    locationmode='USA-states',  
                    scope="usa", 
                    color='count_users',  
                    )
fig.update_layout(title_text='BeerAdvocate - Number of users per states') 
fig.show()
#comment the code below if you don't want to generate the html
fig.write_html('map_users_BA.html', include_plotlyjs='cdn', full_html=False, config={'displayModeBar': False})

### Dealing with missing values

We start by checking the number of missing values in each column of the dataframes. 

In [ ]:
print(RB_US.isnull().sum())
print("Percentage of NaN values in RB_US: ", (RB_US['text'].isnull().sum()/len(RB_US))*100, "%")

US_Code_User           0
location_user          0
US_Code_Beer      604710
beers_location         0
beer_name              0
beer_id                0
brewery_name           0
brewery_id             0
style                  0
abv                25828
date                   0
user_name              0
user_id                0
appearance             0
aroma                  0
palate                 0
taste                  0
overall                0
rating                 0
text                  44
dtype: int64
Percentage of NaN values in RB_US:  0.005057639852362895 %


In [ ]:
print(BA_US.isnull().sum())
print("Percentage of NaN values in BA_US: ", (BA_US['appearance'].isnull().sum()/len(BA_US))*100, "%")

US_Code_User           0
location_user          0
US_Code_Beer      331716
beers_location         0
beer_name              0
beer_id                0
brewery_name           0
brewery_id             0
style                  0
abv                12984
date                   0
user_name              1
user_id                0
appearance          4148
aroma               4148
palate              4148
taste               4148
overall             4148
rating                 0
text                   0
dtype: int64
Percentage of NaN values in BA_US:  0.6316257510834178 %


After our processing, we can see that for `RB_US` we have no NaN values for the rating columns and NaN values for the text column. For `BA_US` we have no NaN values for the text column and NaN values for the rating columns.
If we look at how much these NaN values represent in the dataframes, we can see that for `RB_US` the NaN values represent 0.005% of the data and for `BA_US` the NaN values represent 0.6% of the data. Thus we can drop these rows without losing too much information.

In [ ]:
RB_US = RB_US.dropna(subset=['text'], how='all')
BA_US = BA_US.dropna(subset=['appearance','aroma','palate','taste','overall'], how='all')

### Checking data types

In [ ]:
RB_US.dtypes

US_Code_User       object
location_user      object
US_Code_Beer       object
beers_location     object
beer_name          object
beer_id           float64
brewery_name       object
brewery_id        float64
style              object
abv               float64
date              float64
user_name          object
user_id           float64
appearance        float64
aroma             float64
palate            float64
taste             float64
overall           float64
rating            float64
text               object
dtype: object

In [ ]:
BA_US.dtypes

US_Code_User       object
location_user      object
US_Code_Beer       object
beers_location     object
beer_name          object
beer_id           float64
brewery_name       object
brewery_id        float64
style              object
abv               float64
date              float64
user_name          object
user_id            object
appearance        float64
aroma             float64
palate            float64
taste             float64
overall           float64
rating            float64
text               object
dtype: object

We first have to convert the `date` column to a datetime object. We do this for both `RB_US` and `BA_US`.


In [ ]:
RB_US['date'] = RB_US['date'].apply(datetime.datetime.fromtimestamp)
BA_US['date'] = BA_US['date'].apply(datetime.datetime.fromtimestamp)

We finally have our two dataframes `RB_US` and `BA_US` that we will use for the analysis.

## Merging the dataframes

# Analysis

## Quick statistics from the data

### Number of reviews per brewery
blip-blop-bloop

In [8]:
#sort the breweries with the most reviews
brewery_counts = df_BA['brewery_name'].value_counts().sort_values(ascending=False).head(10)

#plot the repartition of these breweries
fig = px.bar(brewery_counts, x = brewery_counts.index, y = brewery_counts.values, title = 'Top 10 Breweries with the most reviews',color_discrete_sequence=px.colors.sequential.Plasma)

fig.show()
#comment the code below if you don't want to generate the html
fig.write_html('breweries_reviews.html', include_plotlyjs='cdn', full_html=False, config={'displayModeBar': False})

### Number of reviews per state (beer sight)
Most reviewed states according to the beer location, on the BA dataset



In [27]:
#sort the breweries with the most reviews
states_counts = df_BA['US_Code_Beer'].value_counts().sort_values(ascending=False).head(10)

#plot the repartition of these breweries
fig = px.bar(states_counts, x = states_counts.index, y = states_counts.values, title = 'Beer Reviews by State',color_discrete_sequence=px.colors.sequential.Plasma)
fig.update_traces(marker_color='#000FAB')
fig.update_layout(
    xaxis = dict(title = "States"),
    yaxis = dict(title = "Number of reviews")
)
fig.show()
#comment the code below if you don't want to generate the html
fig.write_html('states_reviews_beers_BA.html', include_plotlyjs='cdn', full_html=False, config={'displayModeBar': False})

Most reviewed states according to the beer location, on the BA dataset

In [28]:
#sort the breweries with the most reviews
states_counts = df_RB['US_Code_Beer'].value_counts().sort_values(ascending=False).head(10)

#plot the repartition of these breweries
fig = px.bar(states_counts, x = states_counts.index, y = states_counts.values, title = 'Beer Reviews by State',color_discrete_sequence=px.colors.sequential.Plasma)
fig.update_traces(marker_color='#000FAB')
fig.update_layout(
    xaxis = dict(title = "States"),
    yaxis = dict(title = "Number of reviews")
)
fig.show()
#comment the code below if you don't want to generate the html
fig.write_html('states_reviews_beers_RB.html', include_plotlyjs='cdn', full_html=False, config={'displayModeBar': False})

### Number of reviews per state (user sight)
Most reviewed states according to the user location, according to the BA dataset

In [29]:
#sort the breweries with the most reviews
states_counts = df_BA['US_Code_User'].value_counts().sort_values(ascending=False).head(10)

#plot the repartition of these breweries
fig = px.bar(states_counts, x = states_counts.index, y = states_counts.values, title = "Beer Reviews by State by User",color_discrete_sequence=px.colors.sequential.Plasma)
fig.update_traces(marker_color='#DC3220')
fig.update_layout(
    xaxis = dict(title = "States"),
    yaxis = dict(title = "Number of reviews by User")
)
fig.show()
#comment the code below if you don't want to generate the html
fig.write_html('states_reviews_users_BA.html', include_plotlyjs='cdn', full_html=False, config={'displayModeBar': False})

Most reviewed states according to the user location, according to the BA dataset

In [30]:
#sort the breweries with the most reviews
states_counts = df_RB['US_Code_User'].value_counts().sort_values(ascending=False).head(10)

#plot the repartition of these breweries
fig = px.bar(states_counts, x = states_counts.index, y = states_counts.values, title = "Beer Reviews by State by User",color_discrete_sequence=px.colors.sequential.Plasma)
fig.update_traces(marker_color='#DC3220')
fig.update_layout(
    xaxis = dict(title = "States"),
    yaxis = dict(title = "Number of reviews by User")
)
fig.show()
#comment the code below if you don't want to generate the html
fig.write_html('states_reviews_users_RB.html', include_plotlyjs='cdn', full_html=False, config={'displayModeBar': False})

### Number of reviews per beer
BA

In [34]:
#sort the breweries with the most reviews
beers_counts = df_BA['beer_name'].value_counts().sort_values(ascending=False).head(10)

#plot the repartition of these breweries
fig = px.bar(beers_counts, x = beers_counts.index, y = beers_counts.values, title = "Reviews by Beer",color_discrete_sequence=px.colors.sequential.Plasma)
fig.update_traces(marker_color='#DC3220')
fig.update_layout(
    xaxis = dict(title = "Beer name"),
    yaxis = dict(title = "Number of reviews")
)
fig.show()
#comment the code below if you don't want to generate the html
fig.write_html('beer_reviews_BA.html', include_plotlyjs='cdn', full_html=False, config={'displayModeBar': False})

RB

In [33]:
#sort the breweries with the most reviews
beers_counts = df_RB['beer_name'].value_counts().sort_values(ascending=False).head(10)

#plot the repartition of these breweries
fig = px.bar(beers_counts, x = beers_counts.index, y = beers_counts.values, title = "Reviews by Beer",color_discrete_sequence=px.colors.sequential.Plasma)
fig.update_traces(marker_color='#DC3220')
fig.update_layout(
    xaxis = dict(title = "Beer name"),
    yaxis = dict(title = "Number of reviews")
)
fig.show()
#comment the code below if you don't want to generate the html
fig.write_html('beer_reviews_RB.html', include_plotlyjs='cdn', full_html=False, config={'displayModeBar': False})

Study over the years